###### **PROBABILITAS NAIVE BAYES MENGGUNAKAN PYTHON TANPA MODUL SKLEARN**
-----------------------------------------------------------------
-----------------------------------------------------------------
###### Alfi Hidayatur R.
###### 21083010001 - Data Mining C

In [ ]:
import numpy as np
import pandas as pd
from math import pi, exp, sqrt

data = pd.read_csv("WineQT.csv")

train_data = data.sample(frac=0.8, random_state=1)
test_data = data.drop(train_data.index)

print("Data Latih :", "\n", train_data)
print("\n", "Data Uji :", "\n", test_data)

Data Latih : 
       fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
314             9.3             0.480         0.29             2.1      0.127   
698             7.1             0.340         0.28             2.0      0.082   
846             7.7             0.260         0.26             2.0      0.052   
1136            6.3             0.550         0.15             1.8      0.077   
1097            7.0             0.550         0.13             2.2      0.075   
...             ...               ...          ...             ...        ...   
163             8.0             0.380         0.06             1.8      0.078   
167             8.9             0.635         0.37             1.7      0.263   
145            12.8             0.300         0.74             2.6      0.095   
206             8.7             0.520         0.09             2.5      0.091   
500             7.5             0.380         0.48             2.6      0.073   

      free s

In [ ]:
class_counts = data["quality"].value_counts()
total_samples = len(data)
prior_probs = class_counts / total_samples

print("Probabilitas Prior P(A) :", "\n", prior_probs)

Probabilitas Prior P(A) : 
 5    0.422572
6    0.404199
7    0.125109
4    0.028871
8    0.013998
3    0.005249
Name: quality, dtype: float64


In [ ]:
conditional_probs = {}
for feature in train_data.columns[:-1]:
    for label in set(train_data['quality']):
        subset = train_data[train_data['quality'] == label]
        mean = np.mean(subset[feature])
        std = np.std(subset[feature])
        key = (label, feature)
        conditional_probs[key] = (mean, std)

print("Probabilitas kondisional P(B|A) :" )
for label in set(train_data['quality']):
    print(f"Class: {label}")
    for feature in train_data.columns[:-1]:
        key = (label, feature)
        print(f"\t{feature}: {conditional_probs[key]}")

Probabilitas kondisional P(B|A) :
Class: 3
	fixed acidity: (8.366666666666667, 1.452201394052798)
	volatile acidity: (1.0216666666666667, 0.40935586257219064)
	citric acid: (0.18000000000000002, 0.22015146301277821)
	residual sugar: (3.3000000000000003, 1.6970562748477143)
	chlorides: (0.1396666666666666, 0.048210187950496794)
	free sulfur dioxide: (4.333333333333333, 0.9428090415820634)
	total sulfur dioxide: (13.0, 2.943920288775949)
	density: (0.9974133333333333, 0.0019520473582597462)
	pH: (3.3533333333333335, 0.1426728970602179)
	sulphates: (0.5166666666666667, 0.09392668535736913)
	alcohol: (9.833333333333334, 1.0530379332620874)
	quality: (3.0, 0.0)
Class: 4
	fixed acidity: (7.937037037037038, 1.4611381023653491)
	volatile acidity: (0.7, 0.2158145637484318)
	citric acid: (0.16851851851851854, 0.22016330140760085)
	residual sugar: (2.57037037037037, 1.1161490723530965)
	chlorides: (0.09659259259259256, 0.10225359153676392)
	free sulfur dioxide: (14.333333333333334, 9.906974722292

In [ ]:
prob_features = {}
for col in data.columns[:-1]:
    freq = data[col].value_counts(normalize=True)
    prob_features[col] = freq.to_dict()

print("Nilai Probabilitas Features P(B) :")
for key, value in prob_features.items():
    print(f"\t{key} : {value}")

Nilai Probabilitas Features P(B) :
	fixed acidity : {7.2: 0.03762029746281715, 7.1: 0.03587051618547681, 7.0: 0.03499562554680665, 7.8: 0.03499562554680665, 7.5: 0.03237095363079615, 7.9: 0.03062117235345582, 7.6: 0.02974628171478565, 7.7: 0.02974628171478565, 7.4: 0.02799650043744532, 7.3: 0.02799650043744532, 6.8: 0.02537182852143482, 8.0: 0.02537182852143482, 8.3: 0.024496937882764653, 8.2: 0.024496937882764653, 8.9: 0.023622047244094488, 6.6: 0.021872265966754154, 8.8: 0.021872265966754154, 8.6: 0.020122484689413824, 6.9: 0.020122484689413824, 6.7: 0.020122484689413824, 8.1: 0.019247594050743656, 9.1: 0.019247594050743656, 8.4: 0.017497812773403325, 8.7: 0.017497812773403325, 9.9: 0.017497812773403325, 6.4: 0.015748031496062992, 9.3: 0.015748031496062992, 10.0: 0.014873140857392825, 8.5: 0.014873140857392825, 6.2: 0.01399825021872266, 9.0: 0.01399825021872266, 9.4: 0.011373578302712161, 10.4: 0.011373578302712161, 9.2: 0.010498687664041995, 6.3: 0.010498687664041995, 6.5: 0.0104986

In [ ]:
posteriors = {}
for label in set(test_data["quality"]):
    posteriors[label] = 0
    for i, row in test_data[test_data["quality"] == label].iterrows():
        prior = prior_probs[label]
        likelihood = 1
        for feature in row.keys():
            if feature == "quality" or feature == "Id":
                continue
            mu, sigma = conditional_probs[(label, feature)]
            x = row[feature]
            likelihood *= (1 / (np.sqrt(2 * np.pi) * sigma)) * np.exp(-((x - mu) ** 2) / (2 * sigma ** 2))
        evidence = 1
        for feature in row.keys():
            if feature == "quality" or feature == "Id":
                continue
            evidence *= prob_features[feature][row[feature]]
        posterior = (prior * likelihood) / evidence
        posteriors[label] += np.log(posterior)
print("Probabilitas Naive Bayes P(A|B) :" )
for key, value in posteriors.items():
    print(f"\t{key} : {value}")

Probabilitas Naive Bayes P(A|B) :
	3 : -181.89330479092783
	4 : 218.31137751005042
	5 : 3636.606743525727
	6 : 3496.5842381193747
	7 : 1260.880492146859
	8 : 55.91884354987897


In [ ]:
predictions = []
for i, row in test_data.iterrows():
    max_label = None
    max_posterior = -np.inf
    for label in posteriors:
        if posteriors[label] > max_posterior:
            max_label = label
            max_posterior = posteriors[label]
    predictions.append(max_label)

test_labels = test_data["quality"].tolist()
accuracy = sum([1 for i in range(len(predictions)) if predictions[i] == test_labels[i]]) / len(predictions)
print("\nAkurasi Naive Bayes: {:.2%}".format(accuracy))


Akurasi Naive Bayes: 41.92%
